# ChatFeed 聊天流

PnChatFeed是一个中层布局组件，用于管理一系列聊天消息(ChatMessage)项。该组件提供后端方法来发送消息、流式传输令牌、执行回调、撤销消息以及清除聊天记录。

底层实现为`panel.chat.ChatFeed`，参数基本一致，参考文档：https://panel.holoviz.org/reference/chat/ChatFeed.html


In [1]:
##ignore
%load_ext vuepy
from panel_vuepy import vpanel


## 基本用法

`PnChatFeed`可以不需要任何参数初始化，通过`send`方法发送聊天消息。


In [2]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
<PnCol>
  <PnChatFeed ref="chat_feed" />
  <PnButton name='send' @click='on_click()'/>
</PnCol>
</template>

<script lang='py'>
import panel as pn
from vuepy import ref, onMounted

chat_feed = ref(None)

def on_click():
    message = chat_feed.value.unwrap().send(
        "Hello world!",
        user="Bot",
        avatar="B",
        footer_objects=[pn.widgets.Button(name="Footer Object")]
    )
_ = onMounted(on_click)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n<PnCol>\n  <PnChatFeed ref=\"chat_feed\" />\n  <PnButton name='send' @click='on_click()'/>\n</PnCol>\n</template>\n\n<script lang='py'>\nimport panel as pn\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\n\ndef on_click():\n    message = chat_feed.value.unwrap().send(\n        \"Hello world!\",\n        user=\"Bot\",\n        avatar=\"B\",\n        footer_objects=[pn.widgets.Button(name=\"Footer Object\")]\n    )\n_ = onMounted(on_click)\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] Column
                [0] ChatFeed(_placeholder=ChatMessage, message_params={'show_timestamp': True, ...}, sizing_mode='stretch_width')
                    [0] ChatMessage(object='Hello world!', user='Bot', reactions=[])
                [1] Button(name='send')


## 回调函数

添加回调函数可以使`PnChatFeed`更加有趣。回调函数的签名必须包含最新可用的消息值`contents`。
除了`contents`之外，签名还可以包含最新可用的`user`名称和聊天`instance`。


In [3]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed :callback="echo_message" ref="chat_feed" />
  <PnButton name='send' @click='send_message()'/>
</template>
<script lang='py'>
from vuepy import ref, onMounted

chat_feed = ref(None)

# def echo_message(contents, user, instance):
def echo_message(contents):
    return f"Echoing... {contents}"

def send_message():
    message = chat_feed.value.unwrap().send("Hello!")

_ = onMounted(send_message)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed :callback=\"echo_message\" ref=\"chat_feed\" />\n  <PnButton name='send' @click='send_message()'/>\n</template>\n<script lang='py'>\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\n\n# def echo_message(contents, user, instance):\ndef echo_message(contents):\n    return f\"Echoing... {contents}\"\n\ndef send_message():\n    message = chat_feed.value.unwrap().send(\"Hello!\")\n\n_ = onMounted(send_message)\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] Column
                [0] ChatFeed(_placeholder=ChatMessage, callback=<function setup.<locals>.e..., message_params={'show_timestamp': True, ...}, sizing_mode='stretch_width')
                    [0] ChatMessage(object='Hello!', user='User', reactions=[])
                [1] Button(name='send')


可以更新`callback_user`和`callback_avatar`来分别更改响应者的默认名称和头像。


In [4]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed :callback='echo_message' 
              callback_user='Echo Bot' callback_avatar='🛸' ref="chat_feed" />
  <PnButton name='send' @click='send_message()'/>
</template>

<script lang='py'>
from vuepy import ref, onMounted

chat_feed = ref(None)

def echo_message(contents, user):
    return f"Echoing {user!r}... {contents}"

def send_message():
    message = chat_feed.value.unwrap().send("Hey!")

d = onMounted(send_message)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed :callback='echo_message' \n              callback_user='Echo Bot' callback_avatar='\ud83d\udef8' ref=\"chat_feed\" />\n  <PnButton name='send' @click='send_message()'/>\n</template>\n\n<script lang='py'>\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\n\ndef echo_message(contents, user):\n    return f\"Echoing {user!r}... {contents}\"\n\ndef send_message():\n    message = chat_feed.value.unwrap().send(\"Hey!\")\n\nd = onMounted(send_message)\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] Column
                [0] ChatFeed(_placeholder=ChatMessage, callback=<function setup.<locals>.e..., callback_avatar='🛸', callback_user='Echo Bot', message_params={'show_timestamp': True, ...}, sizing_mode='stretch_width')
                    [0] ChatMessage(object='Hey!', user='User', reactions=[])
                [1] Button(name='send')


指定的`callback`也可以返回一个包含`value`、`user`和`avatar`键的字典，这将覆盖默认的`callback_user`和`callback_avatar`。


In [5]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed :callback="parrot_message" 
              callback_user='Echo Bot' 
              callback_avatar='🛸' ref="chat_feed" />
</template>

<script lang='py'>
from vuepy import ref, onMounted

chat_feed = ref(None)

def parrot_message(contents):
    return {"value": f"No, {contents.lower()}", "user": "Parrot", "avatar": "🦜"}

def send_message():
    message = chat_feed.value.unwrap().send("Are you a parrot?")

_ = onMounted(send_message)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed :callback=\"parrot_message\" \n              callback_user='Echo Bot' \n              callback_avatar='\ud83d\udef8' ref=\"chat_feed\" />\n</template>\n\n<script lang='py'>\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\n\ndef parrot_message(contents):\n    return {\"value\": f\"No, {contents.lower()}\", \"user\": \"Parrot\", \"avatar\": \"\ud83e\udd9c\"}\n\ndef send_message():\n    message = chat_feed.value.unwrap().send(\"Are you a parrot?\")\n\n_ = onMounted(send_message)\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] ChatFeed(_placeholder=ChatMessage, callback=<function setup.<locals>.p..., callback_avatar='🛸', callback_user='Echo Bot', message_params={'show_timestamp': True, ...}, sizing_mode='stretch_width')
                [0] ChatMessage(object='Are you a parrot?', user='User', reactions=[])


如果不希望与`send`一起触发回调，请将`respond`设置为`False`。


In [6]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed :callback="parrot_message" callback_user='Echo Bot' callback_avatar='🛸' ref="chat_feed" />
</template>

<script lang='py'>
from vuepy import ref, onMounted

chat_feed = ref(None)

def parrot_message(contents):
    return {"value": f"No, {contents.lower()}", "user": "Parrot", "avatar": "🦜"}

def send_message():
    message = chat_feed.value.unwrap().send("Don't parrot this.", respond=False)

_ = onMounted(send_message)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed :callback=\"parrot_message\" callback_user='Echo Bot' callback_avatar='\ud83d\udef8' ref=\"chat_feed\" />\n</template>\n\n<script lang='py'>\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\n\ndef parrot_message(contents):\n    return {\"value\": f\"No, {contents.lower()}\", \"user\": \"Parrot\", \"avatar\": \"\ud83e\udd9c\"}\n\ndef send_message():\n    message = chat_feed.value.unwrap().send(\"Don't parrot this.\", respond=False)\n\n_ = onMounted(send_message)\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] ChatFeed(_placeholder=ChatMessage, callback=<function setup.<locals>.p..., callback_avatar='🛸', callback_user='Echo Bot', message_params={'show_timestamp': True, ...}, sizing_mode='stretch_width')
                [0] ChatMessage(object="Don't parrot this.", user='User', reactions=[])


可以通过将`callback_exception`设置为`"summary"`来显示异常。


In [7]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed :callback="bad_callback" callback_exception='summary' ref="chat_feed" />
</template>

<script lang='py'>
from vuepy import ref, onMounted

chat_feed = ref(None)

def bad_callback(contents):
    return 1 / 0

def send_message():
    chat_feed.value.unwrap().send("This will fail...")

_ = onMounted(send_message)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed :callback=\"bad_callback\" callback_exception='summary' ref=\"chat_feed\" />\n</template>\n\n<script lang='py'>\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\n\ndef bad_callback(contents):\n    return 1 / 0\n\ndef send_message():\n    chat_feed.value.unwrap().send(\"This will fail...\")\n\n_ = onMounted(send_message)\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] ChatFeed(_placeholder=ChatMessage, callback=<function setup.<locals>.b..., message_params={'show_timestamp': True, ...}, sizing_mode='stretch_width')
                [0] ChatMessage(object='This will fail...', user='User', reactions=[])


## 异步回调

`PnChatFeed`还支持*异步*`callback`。我们建议尽可能使用*异步*`callback`以保持应用程序的快速响应，*只要函数中没有阻塞事件循环的内容*。


In [8]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed :callback="parrot_message" callback_user='Echo Bot' ref="chat_feed" />
</template>

<script lang='py'>
import asyncio
from vuepy import ref, onMounted

chat_feed = ref(None)

async def parrot_message(contents):
    await asyncio.sleep(2.8)
    return {"value": f"No, {contents.lower()}", "user": "Parrot", "avatar": "🦜"}

def send_message():
    message = chat_feed.value.unwrap().send("Are you a parrot?")

_ = onMounted(send_message)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed :callback=\"parrot_message\" callback_user='Echo Bot' ref=\"chat_feed\" />\n</template>\n\n<script lang='py'>\nimport asyncio\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\n\nasync def parrot_message(contents):\n    await asyncio.sleep(2.8)\n    return {\"value\": f\"No, {contents.lower()}\", \"user\": \"Parrot\", \"avatar\": \"\ud83e\udd9c\"}\n\ndef send_message():\n    message = chat_feed.value.unwrap().send(\"Are you a parrot?\")\n\n_ = onMounted(send_message)\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] ChatFeed(_placeholder=ChatMessage, callback=<function setup.<locals>.p..., callback_user='Echo Bot', message_params={'show_timestamp': True, ...}, sizing_mode='stretch_width')
                [0] ChatMessage(object='Are you a parrot?', user='User', reactions=[])


流式输出的最简单和最优方式是通过*异步生成器*。如果您不熟悉这个术语，只需在函数前加上`async`，并用`yield`替换`return`。


In [9]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed :callback="stream_message" ref="chat_feed" />
</template>

<script lang='py'>
import asyncio
from vuepy import ref, onMounted

chat_feed = ref(None)

async def stream_message(contents):
    message = ""
    for character in contents:
        message += character
        await asyncio.sleep(0.1)
        yield message

def send_message():
    message = chat_feed.value.unwrap().send("Streaming...")

_ = onMounted(send_message)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed :callback=\"stream_message\" ref=\"chat_feed\" />\n</template>\n\n<script lang='py'>\nimport asyncio\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\n\nasync def stream_message(contents):\n    message = \"\"\n    for character in contents:\n        message += character\n        await asyncio.sleep(0.1)\n        yield message\n\ndef send_message():\n    message = chat_feed.value.unwrap().send(\"Streaming...\")\n\n_ = onMounted(send_message)\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] ChatFeed(_placeholder=ChatMessage, callback=<function setup.<locals>.s..., message_params={'show_timestamp': True, ...}, sizing_mode='stretch_width')
                [0] ChatMessage(object='Streaming...', user='User', reactions=[])


如果不连接字符，也可以持续替换原始消息。


In [10]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed :callback="replace_message" ref="chat_feed" />
</template>

<script lang='py'>
import asyncio
from vuepy import ref, onMounted

chat_feed = ref(None)

async def replace_message(contents):
    for character in contents:
        await asyncio.sleep(0.1)
        yield character

def send_message():
    message = chat_feed.value.unwrap().send("ABCDEFGHIJKLMNOPQRSTUVWXYZ")

_ = onMounted(send_message)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed :callback=\"replace_message\" ref=\"chat_feed\" />\n</template>\n\n<script lang='py'>\nimport asyncio\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\n\nasync def replace_message(contents):\n    for character in contents:\n        await asyncio.sleep(0.1)\n        yield character\n\ndef send_message():\n    message = chat_feed.value.unwrap().send(\"ABCDEFGHIJKLMNOPQRSTUVWXYZ\")\n\n_ = onMounted(send_message)\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] ChatFeed(_placeholder=ChatMessage, callback=<function setup.<locals>.r..., message_params={'show_timestamp': True, ...}, sizing_mode='stretch_width')
                [0] ChatMessage(object='ABCDEFGHIJKLMNOPQRSTUVWXYZ', user='User', reactions=[])


也可以手动触发回调与`respond`。这对于从初始消息实现一系列响应很有用！


In [11]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed :callback="chain_message" ref="chat_feed" />
</template>

<script lang='py'>
import asyncio
from vuepy import ref, onMounted

chat_feed = ref(None)

async def chain_message(contents, user, instance):
    await asyncio.sleep(1.8)
    if user == "User":
        yield {"user": "Bot 1", "value": "Hi User! I'm Bot 1--here to greet you."}
        instance.respond()
    elif user == "Bot 1":
        yield {
            "user": "Bot 2",
            "value": "Hi User; I see that Bot 1 already greeted you; I'm Bot 2.",
        }
        instance.respond()
    elif user == "Bot 2":
        yield {
            "user": "Bot 3",
            "value": "I'm Bot 3; the last bot that will respond. See ya!",
        }

def send_message():
    message = chat_feed.value.unwrap().send("Hello bots!")

_ = onMounted(send_message)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed :callback=\"chain_message\" ref=\"chat_feed\" />\n</template>\n\n<script lang='py'>\nimport asyncio\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\n\nasync def chain_message(contents, user, instance):\n    await asyncio.sleep(1.8)\n    if user == \"User\":\n        yield {\"user\": \"Bot 1\", \"value\": \"Hi User! I'm Bot 1--here to greet you.\"}\n        instance.respond()\n    elif user == \"Bot 1\":\n        yield {\n            \"user\": \"Bot 2\",\n            \"value\": \"Hi User; I see that Bot 1 already greeted you; I'm Bot 2.\",\n        }\n        instance.respond()\n    elif user == \"Bot 2\":\n        yield {\n            \"user\": \"Bot 3\",\n            \"value\": \"I'm Bot 3; the last bot that will respond. See ya!\",\n        }\n\ndef send_message():\n    message = chat_feed.value.unwrap().send(\"Hello bots!\")\n\n_ = onMounted(send_message)\n</script>\n", "setup": ""

Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] ChatFeed(_placeholder=ChatMessage, callback=<function setup.<locals>.c..., message_params={'show_timestamp': True, ...}, sizing_mode='stretch_width')
                [0] ChatMessage(object='Hello bots!', user='User', reactions=[])


## 编辑回调

可以将`edit_callback`附加到`PnChatFeed`以处理消息编辑。签名必须包含最新可用的消息值`contents`、编辑消息的索引和聊天`instance`。


In [12]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed :callback="echo_callback" :edit_callback="edit_callback" callback_user="Echo Guy" ref="chat_feed" />
</template>

<script lang='py'>
from vuepy import ref, onMounted

chat_feed = ref(None)

def echo_callback(content):
    return content

def edit_callback(content, index, instance):
    instance.objects[index + 1].object = content

def send_message():
    chat_feed.value.unwrap().send("Edit this")

_ = onMounted(send_message)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed :callback=\"echo_callback\" :edit_callback=\"edit_callback\" callback_user=\"Echo Guy\" ref=\"chat_feed\" />\n</template>\n\n<script lang='py'>\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\n\ndef echo_callback(content):\n    return content\n\ndef edit_callback(content, index, instance):\n    instance.objects[index + 1].object = content\n\ndef send_message():\n    chat_feed.value.unwrap().send(\"Edit this\")\n\n_ = onMounted(send_message)\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] ChatFeed(_placeholder=ChatMessage, callback=<function setup.<locals>.e..., callback_user='Echo Guy', edit_callback=<function setup.<locals>.e..., message_params={'show_timestamp': True, ...}, sizing_mode='stretch_width')
                [0] ChatMessage(object='Edit this', user='User', reactions=[])


## 步骤

可以通过一系列`ChatStep`提供中间步骤，如思想链。


In [13]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed ref="chat_feed" />
</template>

<script lang='py'>
import time
from vuepy import ref, onMounted

chat_feed = ref(None)

def demo_steps():
    # First step
    with chat_feed.value.unwrap().add_step(
        "To answer the user's query, I need to first create a plan.", 
        title="Create a plan", user='Agent'
    ) as step:
        step.stream("\n\n...Okay the plan is to demo this!")
    
    # Second step - append to existing message
    with chat_feed.value.unwrap().add_step(
        title="Execute the plan", status="running"
    ) as step:
        step.stream("\n\n...Executing plan...")
        time.sleep(1)
        step.stream("\n\n...Handing over to SQL Agent")
    
    # Third step - new user creates a new message
    with chat_feed.value.unwrap().add_step(
        title="Running SQL query", user='SQL Agent'
    ) as step:
        step.stream('Querying...')
        time.sleep(1)
        step.stream('\nSELECT * FROM TABLE')

_ = onMounted(demo_steps)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed ref=\"chat_feed\" />\n</template>\n\n<script lang='py'>\nimport time\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\n\ndef demo_steps():\n    # First step\n    with chat_feed.value.unwrap().add_step(\n        \"To answer the user's query, I need to first create a plan.\", \n        title=\"Create a plan\", user='Agent'\n    ) as step:\n        step.stream(\"\\n\\n...Okay the plan is to demo this!\")\n    \n    # Second step - append to existing message\n    with chat_feed.value.unwrap().add_step(\n        title=\"Execute the plan\", status=\"running\"\n    ) as step:\n        step.stream(\"\\n\\n...Executing plan...\")\n        time.sleep(1)\n        step.stream(\"\\n\\n...Handing over to SQL Agent\")\n    \n    # Third step - new user creates a new message\n    with chat_feed.value.unwrap().add_step(\n        title=\"Running SQL query\", user='SQL Agent'\n    ) as step:\n        ste

Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] ChatFeed(_placeholder=ChatMessage, message_params={'show_timestamp': True, ...}, sizing_mode='stretch_width')
                [0] ChatMessage(object=Card(css_classes=['chat-steps'], header=HTML, min_width=100, styles={'margin-inline': '10px'}, stylesheets=['https://cdn.holoviz.org/...])
    [0] ChatStep(collapsed=True, context_exception='summary', default_title='Create a plan', header=Row, margin=(5, 1), status='success', stylesheets=['https://cdn.holoviz.org/...], title='Create a plan')
        [0] Markdown(str, css_classes=['step-message'], stylesheets=['https://cdn.holoviz.org/...])
    [1] ChatStep(collapsed=True, context_exception='summary', default_title='Execute the plan', header=Row, margin=(5, 1), status='success', title='Execute the plan')
        [0] Markdown(str, css_classes=['step-message']), user='Agent', reactions=[])
                [1] ChatMessage(object=Card(css_classes=['chat-steps'], header=HTML, min_width=100, styles={'margin-inline': '10px'}, stylesheets=['https://cdn.holoviz.org/...])
    [0] ChatStep(collapsed=True, context_exception='summary', default_title='Running SQL query', header=Row, margin=(5, 1), status='success', stylesheets=['https://cdn.holoviz.org/...], title='Running SQL query')
        [0] Markdown(str, css_classes=['step-message']), user='SQL Agent', reactions=[])


## 提示用户

可以使用`prompt_user`暂时暂停代码执行并提示用户回答问题或填写表单，该方法接受任何Panel `component`和后续`callback`（带有`component`和`instance`作为args）在提交后执行。


In [14]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed :callback="show_interest" callback_user="Ice Cream Bot" ref="chat_feed" />
</template>

<script lang='py'>
from vuepy import ref, onMounted

chat_feed = ref(None)

def repeat_answer(component, instance):
    contents = component.value
    instance.send(f"Wow, {contents}, that's my favorite flavor too!", respond=False, user="Ice Cream Bot")

def show_interest(contents, user, instance):
    if "ice" in contents or "cream" in contents:
        answer_input = {"component": "PnTextInput", "props": {"placeholder": "Enter your favorite ice cream flavor"}}
        instance.prompt_user(answer_input, callback=repeat_answer)
    else:
        return "I'm not interested in that topic."

def send_message():
    chat_feed.value.unwrap().send("ice cream")

_ = onMounted(send_message)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed :callback=\"show_interest\" callback_user=\"Ice Cream Bot\" ref=\"chat_feed\" />\n</template>\n\n<script lang='py'>\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\n\ndef repeat_answer(component, instance):\n    contents = component.value\n    instance.send(f\"Wow, {contents}, that's my favorite flavor too!\", respond=False, user=\"Ice Cream Bot\")\n\ndef show_interest(contents, user, instance):\n    if \"ice\" in contents or \"cream\" in contents:\n        answer_input = {\"component\": \"PnTextInput\", \"props\": {\"placeholder\": \"Enter your favorite ice cream flavor\"}}\n        instance.prompt_user(answer_input, callback=repeat_answer)\n    else:\n        return \"I'm not interested in that topic.\"\n\ndef send_message():\n    chat_feed.value.unwrap().send(\"ice cream\")\n\n_ = onMounted(send_message)\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] ChatFeed(_placeholder=ChatMessage, callback=<function setup.<locals>.s..., callback_user='Ice Cream Bot', message_params={'show_timestamp': True, ...}, sizing_mode='stretch_width')
                [0] ChatMessage(object='ice cream', user='User', reactions=[])


还可以设置一个`predicate`来评估组件的状态，例如小部件是否有值。如果提供，当谓词返回`True`时，提交按钮将被启用。


In [15]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed :callback="show_interest" callback_user="Ice Cream Bot" ref="chat_feed" />
</template>

<script lang='py'>
from vuepy import ref, onMounted

chat_feed = ref(None)

def is_chocolate(component):
    return "chocolate" in component.value.lower()

def repeat_answer(component, instance):
    contents = component.value
    instance.send(f"Wow, {contents}, that's my favorite flavor too!", respond=False, user="Ice Cream Bot")

def show_interest(contents, user, instance):
    if "ice" in contents or "cream" in contents:
        answer_input = {"component": "PnTextInput", "props": {"placeholder": "Enter your favorite ice cream flavor"}}
        instance.prompt_user(answer_input, callback=repeat_answer, predicate=is_chocolate)
    else:
        return "I'm not interested in that topic."

def send_message():
    chat_feed.value.unwrap().send("ice cream")

_ = onMounted(send_message)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed :callback=\"show_interest\" callback_user=\"Ice Cream Bot\" ref=\"chat_feed\" />\n</template>\n\n<script lang='py'>\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\n\ndef is_chocolate(component):\n    return \"chocolate\" in component.value.lower()\n\ndef repeat_answer(component, instance):\n    contents = component.value\n    instance.send(f\"Wow, {contents}, that's my favorite flavor too!\", respond=False, user=\"Ice Cream Bot\")\n\ndef show_interest(contents, user, instance):\n    if \"ice\" in contents or \"cream\" in contents:\n        answer_input = {\"component\": \"PnTextInput\", \"props\": {\"placeholder\": \"Enter your favorite ice cream flavor\"}}\n        instance.prompt_user(answer_input, callback=repeat_answer, predicate=is_chocolate)\n    else:\n        return \"I'm not interested in that topic.\"\n\ndef send_message():\n    chat_feed.value.unwrap().send(\"ice cream\")\

Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] ChatFeed(_placeholder=ChatMessage, callback=<function setup.<locals>.s..., callback_user='Ice Cream Bot', message_params={'show_timestamp': True, ...}, sizing_mode='stretch_width')
                [0] ChatMessage(object='ice cream', user='User', reactions=[])


## 序列化

聊天历史可以通过`serialize`并设置`format="transformers"`来序列化，以供`transformers`或`openai`包使用。


In [16]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed ref="chat_feed" />
  <PnCol>
    <PnButton @click="send_messages()" name="Send Messages" />
    <PnButton @click="serialize_chat()" name="Serialize" />
    <PnTextAreaInput v-model="serialized.value" :rows="10" />
  </PnCol>
</template>

<script lang='py'>
from vuepy import ref, onMounted

chat_feed = ref(None)
serialized = ref("")

def send_messages():
    chat_feed.value.unwrap().send("Hello!", user="User")
    chat_feed.value.unwrap().send("Hi there!", user="Bot 1")
    chat_feed.value.unwrap().send("How are you?", user="User")
    chat_feed.value.unwrap().send("I'm doing well!", user="Bot 2")

def serialize_chat():
    serialized.value = str(chat_feed.value.unwrap().serialize(format="transformers"))
  
m1 = onMounted(send_messages)
m2 = onMounted(serialize_chat)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed ref=\"chat_feed\" />\n  <PnCol>\n    <PnButton @click=\"send_messages()\" name=\"Send Messages\" />\n    <PnButton @click=\"serialize_chat()\" name=\"Serialize\" />\n    <PnTextAreaInput v-model=\"serialized.value\" :rows=\"10\" />\n  </PnCol>\n</template>\n\n<script lang='py'>\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\nserialized = ref(\"\")\n\ndef send_messages():\n    chat_feed.value.unwrap().send(\"Hello!\", user=\"User\")\n    chat_feed.value.unwrap().send(\"Hi there!\", user=\"Bot 1\")\n    chat_feed.value.unwrap().send(\"How are you?\", user=\"User\")\n    chat_feed.value.unwrap().send(\"I'm doing well!\", user=\"Bot 2\")\n\ndef serialize_chat():\n    serialized.value = str(chat_feed.value.unwrap().serialize(format=\"transformers\"))\n  \nm1 = onMounted(send_messages)\nm2 = onMounted(serialize_chat)\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] Column
                [0] ChatFeed(_placeholder=ChatMessage, message_params={'show_timestamp': True, ...}, sizing_mode='stretch_width')
                    [0] ChatMessage(object='Hello!', user='User', reactions=[])
                    [1] ChatMessage(object='Hi there!', user='Bot 1', reactions=[])
                    [2] ChatMessage(object='How are you?', user='User', reactions=[])
                    [3] ChatMessage(object="I'm doing well!", user='Bot 2', reactions=[])
                [1] Column
                    [0] Button(name='Send Messages')
                    [1] Button(name='Serialize')
                    [2] TextAreaInput(rows=10, value='[{\'role\': \'user\', ...)


可以设置`role_names`来显式映射角色到ChatMessage的用户名。


In [17]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed ref="chat_feed" />
  <PnCol>
    <PnButton @click="send_messages()" name="Send Messages" />
    <PnButton @click="serialize_chat()" name="Serialize with role_names" />
    <PnTextAreaInput v-model="serialized.value" :rows="10" />
  </PnCol>
</template>

<script lang='py'>
from vuepy import ref, onMounted

chat_feed = ref(None)
serialized = ref("")

def send_messages():
    chat_feed.value.unwrap().send("Hello!", user="User")
    chat_feed.value.unwrap().send("Hi there!", user="Bot 1")
    chat_feed.value.unwrap().send("How are you?", user="User")
    chat_feed.value.unwrap().send("I'm doing well!", user="Bot 2")

def serialize_chat():
    serialized.value = str(chat_feed.value.unwrap().serialize(
        format="transformers", 
        role_names={"assistant": ["Bot 1", "Bot 2", "Bot 3"]}
    ))

m1 = onMounted(send_messages)
m2 = onMounted(serialize_chat)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed ref=\"chat_feed\" />\n  <PnCol>\n    <PnButton @click=\"send_messages()\" name=\"Send Messages\" />\n    <PnButton @click=\"serialize_chat()\" name=\"Serialize with role_names\" />\n    <PnTextAreaInput v-model=\"serialized.value\" :rows=\"10\" />\n  </PnCol>\n</template>\n\n<script lang='py'>\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\nserialized = ref(\"\")\n\ndef send_messages():\n    chat_feed.value.unwrap().send(\"Hello!\", user=\"User\")\n    chat_feed.value.unwrap().send(\"Hi there!\", user=\"Bot 1\")\n    chat_feed.value.unwrap().send(\"How are you?\", user=\"User\")\n    chat_feed.value.unwrap().send(\"I'm doing well!\", user=\"Bot 2\")\n\ndef serialize_chat():\n    serialized.value = str(chat_feed.value.unwrap().serialize(\n        format=\"transformers\", \n        role_names={\"assistant\": [\"Bot 1\", \"Bot 2\", \"Bot 3\"]}\n    ))\n\nm1 = onMounted(send_messages)\nm

Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] Column
                [0] ChatFeed(_placeholder=ChatMessage, message_params={'show_timestamp': True, ...}, sizing_mode='stretch_width')
                    [0] ChatMessage(object='Hello!', user='User', reactions=[])
                    [1] ChatMessage(object='Hi there!', user='Bot 1', reactions=[])
                    [2] ChatMessage(object='How are you?', user='User', reactions=[])
                    [3] ChatMessage(object="I'm doing well!", user='Bot 2', reactions=[])
                [1] Column
                    [0] Button(name='Send Messages')
                    [1] Button(name='Serialize with r...)
                    [2] TextAreaInput(rows=10, value='[{\'role\': \'assistant\'...)


## 流式传输

如果返回的对象不是生成器（特别是LangChain输出），仍然可以使用`stream`方法流式传输输出。


In [18]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed ref="chat_feed" />
</template>

<script lang='py'>
import time
import panel as pn
from vuepy import ref, onMounted

chat_feed = ref(None)

def demo_stream():
    # Create a new message
    message = chat_feed.value.unwrap().stream("Hello", user="Aspiring User", avatar="🤓")
    
    # Stream (append) to the previous message
    message = chat_feed.value.unwrap().stream(
        " World!",
        user="Aspiring User",
        avatar="🤓",
        message=message,
        footer_objects=[pn.widgets.Button(name="Footer Object")]
    )
    
    # Demonstrate streaming with a loop
    message = None
    for n in "12":
        time.sleep(0.1)
        message = chat_feed.value.unwrap().stream(n, message=message)

@onMounted
def demo():
    pn.state.add_periodic_callback(demo_stream, 500, count=1)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed ref=\"chat_feed\" />\n</template>\n\n<script lang='py'>\nimport time\nimport panel as pn\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\n\ndef demo_stream():\n    # Create a new message\n    message = chat_feed.value.unwrap().stream(\"Hello\", user=\"Aspiring User\", avatar=\"\ud83e\udd13\")\n    \n    # Stream (append) to the previous message\n    message = chat_feed.value.unwrap().stream(\n        \" World!\",\n        user=\"Aspiring User\",\n        avatar=\"\ud83e\udd13\",\n        message=message,\n        footer_objects=[pn.widgets.Button(name=\"Footer Object\")]\n    )\n    \n    # Demonstrate streaming with a loop\n    message = None\n    for n in \"12\":\n        time.sleep(0.1)\n        message = chat_feed.value.unwrap().stream(n, message=message)\n\n@onMounted\ndef demo():\n    pn.state.add_periodic_callback(demo_stream, 500, count=1)\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] ChatFeed(_placeholder=ChatMessage, message_params={'show_timestamp': True, ...}, sizing_mode='stretch_width')


## 自定义

可以通过`message_params`传递`ChatEntry`参数。


In [19]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed 
    :message_params="message_params"
    ref="chat_feed" />
</template>

<script lang='py'>
from vuepy import ref, onMounted

chat_feed = ref(None)
message_params = {
    "default_avatars": {"System": "S", "User": "👤"}, 
    "reaction_icons": {"like": "thumb-up"}
}

def send_messages():
    chat_feed.value.unwrap().send(user="System", value="This is the System speaking.")
    chat_feed.value.unwrap().send(user="User", value="This is the User speaking.")

m1 = onMounted(send_messages)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed \n    :message_params=\"message_params\"\n    ref=\"chat_feed\" />\n</template>\n\n<script lang='py'>\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\nmessage_params = {\n    \"default_avatars\": {\"System\": \"S\", \"User\": \"\ud83d\udc64\"}, \n    \"reaction_icons\": {\"like\": \"thumb-up\"}\n}\n\ndef send_messages():\n    chat_feed.value.unwrap().send(user=\"System\", value=\"This is the System speaking.\")\n    chat_feed.value.unwrap().send(user=\"User\", value=\"This is the User speaking.\")\n\nm1 = onMounted(send_messages)\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] ChatFeed(_placeholder=ChatMessage, message_params={'default_avatars': {'Syst...}, sizing_mode='stretch_width')
                [0] ChatMessage(object='This is the System speaking.', user='System', reactions=[])
                [1] ChatMessage(object='This is the User speaking.', user='User', reactions=[])


直接将这些参数传递给ChatFeed构造函数，它将自动转发到`message_params`中。


In [20]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed 
    :default_avatars='{"System": "S", "User": "👤"}'
    :reaction_icons='{"like": "thumb-up"}'
    ref="chat_feed" />
</template>

<script lang='py'>
from vuepy import ref, onMounted

chat_feed = ref(None)

def send_messages():
    chat_feed.value.unwrap().send(user="System", value="This is the System speaking.")
    chat_feed.value.unwrap().send(user="User", value="This is the User speaking.")

m1 = onMounted(send_messages)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed \n    :default_avatars='{\"System\": \"S\", \"User\": \"\ud83d\udc64\"}'\n    :reaction_icons='{\"like\": \"thumb-up\"}'\n    ref=\"chat_feed\" />\n</template>\n\n<script lang='py'>\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\n\ndef send_messages():\n    chat_feed.value.unwrap().send(user=\"System\", value=\"This is the System speaking.\")\n    chat_feed.value.unwrap().send(user=\"User\", value=\"This is the User speaking.\")\n\nm1 = onMounted(send_messages)\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] ChatFeed(_placeholder=ChatMessage, message_params={'default_avatars': {'Syst...}, sizing_mode='stretch_width')
                [0] ChatMessage(object='This is the System speaking.', user='System', reactions=[])
                [1] ChatMessage(object='This is the User speaking.', user='User', reactions=[])


也可以通过设置`message_params`参数来自定义聊天流的外观。


In [21]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnChatFeed 
    :show_activity_dot="True"
    :message_params="message_params"
    ref="chat_feed" />
</template>

<script lang='py'>
from vuepy import ref, onMounted

chat_feed = ref(None)
message_params = {
    "stylesheets": [
        """
        .message {
            background-color: tan;
            font-family: "Courier New";
            font-size: 24px;
        }
        """
    ]
}

def send_message():
    chat_feed.value.unwrap().send("I am so stylish!")

m1 = onMounted(send_message)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnChatFeed \n    :show_activity_dot=\"True\"\n    :message_params=\"message_params\"\n    ref=\"chat_feed\" />\n</template>\n\n<script lang='py'>\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\nmessage_params = {\n    \"stylesheets\": [\n        \"\"\"\n        .message {\n            background-color: tan;\n            font-family: \"Courier New\";\n            font-size: 24px;\n        }\n        \"\"\"\n    ]\n}\n\ndef send_message():\n    chat_feed.value.unwrap().send(\"I am so stylish!\")\n\nm1 = onMounted(send_message)\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] ChatFeed(_placeholder=ChatMessage, message_params={'stylesheets': ['\n      ...}, sizing_mode='stretch_width')
                [0] ChatMessage(object='I am so stylish!', user='User', reactions=[])


## 自定义聊天界面

您也可以在`PnChatFeed`的基础上构建自己的自定义聊天界面。


In [22]:
%%vuepy_run --plugins vpanel --show-code --backend='panel'
<template>
  <PnCol>
    <PnChatFeed
      ref="chat_feed"
      :callback="get_response"
      :height="500"
      :message_params="message_params"
    />
    <PnLayout.Divider />
    <PnRow>
      <span>Click a button</span>
      <PnButton name="Andrew" @click="send_andrew()" />
      <PnButton name="Marc" @click="send_marc()" />
      <PnButton name="Undo" @click="undo_messages()" />
      <PnButton name="Clear" @click="clear_messages()" />
    </PnRow>
  </PnCol>
</template>

<script lang='py'>
import asyncio
from vuepy import ref, onMounted

chat_feed = ref(None)
ASSISTANT_AVATAR = "https://upload.wikimedia.org/wikipedia/commons/6/63/Yumi_UBports.png"

message_params = {
    "default_avatars": {"Assistant": ASSISTANT_AVATAR}
}

async def get_response(contents, user):
    await asyncio.sleep(0.88)
    return {
        "Marc": "It is 2",
        "Andrew": "It is 4",
    }.get(user, "I don't know")

def send_marc():
    chat_feed.value.unwrap().send(
        "What is the square root of 4?", user="Marc", avatar="🚴"
    )

def send_andrew():
    chat_feed.value.unwrap().send(
        "What is the square root of 4 squared?", user="Andrew", avatar="🏊"
    )

def undo_messages():
    chat_feed.value.unwrap().undo(2)

def clear_messages():
    chat_feed.value.unwrap().clear()

def init_chat():
    chat_feed.value.unwrap().send("Hi There!", user="Assistant", avatar=ASSISTANT_AVATAR)

m1 = onMounted(init_chat)
</script>

{"vue": "<!-- --plugins vpanel --show-code --backend='panel' -->\n<template>\n  <PnCol>\n    <PnChatFeed\n      ref=\"chat_feed\"\n      :callback=\"get_response\"\n      :height=\"500\"\n      :message_params=\"message_params\"\n    />\n    <PnLayout.Divider />\n    <PnRow>\n      <span>Click a button</span>\n      <PnButton name=\"Andrew\" @click=\"send_andrew()\" />\n      <PnButton name=\"Marc\" @click=\"send_marc()\" />\n      <PnButton name=\"Undo\" @click=\"undo_messages()\" />\n      <PnButton name=\"Clear\" @click=\"clear_messages()\" />\n    </PnRow>\n  </PnCol>\n</template>\n\n<script lang='py'>\nimport asyncio\nfrom vuepy import ref, onMounted\n\nchat_feed = ref(None)\nASSISTANT_AVATAR = \"https://upload.wikimedia.org/wikipedia/commons/6/63/Yumi_UBports.png\"\n\nmessage_params = {\n    \"default_avatars\": {\"Assistant\": ASSISTANT_AVATAR}\n}\n\nasync def get_response(contents, user):\n    await asyncio.sleep(0.88)\n    return {\n        \"Marc\": \"It is 2\",\n        \"An

Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] Column
                [0] ChatFeed(_placeholder=ChatMessage, callback=<function setup.<locals>.g..., height=500, message_params={'default_avatars': {'Assi...}, sizing_mode='stretch_width')
                    [0] ChatMessage(object='Hi There!', user='Assistant', reactions=[])
                [1] HTML(str)
                [2] Row
                    [0] HTML(str)
                    [1] Button(name='Andrew')
                    [2] Button(name='Marc')
                    [3] Button(name='Undo')
                    [4] Button(name='Clear')


## API

### 属性

| 属性名    | 说明                 | 类型                                                           | 默认值 |
| -------- | ------------------- | ---------------------------------------------------------------| ------- |
| objects | 添加到聊天流的消息 | ^[List[ChatMessage]] | [] |
| renderers | 接受值并返回Panel对象的可调用对象或可调用对象列表 | ^[List[Callable]] | None |
| callback | 当用户发送消息或调用`respond`时执行的回调 | ^[callable] | None |
| card_params | 传递给Card的参数 | ^[Dict] | {} |
| message_params | 传递给每个ChatMessage的参数 | ^[Dict] | {} |
| header | 聊天流的标题 | ^[Any] | None |
| callback_user | 回调提供的消息的默认用户名 | ^[str] | "AI" |
| callback_avatar | 回调提供的条目的默认头像 | ^[str, BytesIO, bytes, ImageBase] | None |
| callback_exception | 如何处理回调引发的异常 | ^[str, Callable] | "raise" |
| edit_callback | 当用户编辑消息时执行的回调 | ^[callable] | None |
| help_text | 初始化聊天记录中的聊天消息 | ^[str] | None |
| placeholder_text | 显示在占位符图标旁边的文本 | ^[str] | "Thinking..." |
| placeholder_params | 传递给占位符`ChatMessage`的参数 | ^[dict] | {} |
| placeholder_threshold | 显示占位符前缓冲的最小持续时间（秒） | ^[float] | 0.2 |
| post_hook | 在新消息完全添加后执行的钩子 | ^[callable] | None |
| auto_scroll_limit | 从Column中最新对象到激活更新时自动滚动的最大像素距离 | ^[int] | 10 |
| scroll_button_threshold | 从Column中最新对象到显示滚动按钮的最小像素距离 | ^[int] | 100 |
| load_buffer | 在可见对象每侧加载的对象数 | ^[int] | 10 |
| show_activity_dot | 是否在流式传输回调响应时在ChatMessage上显示活动点 | ^[bool] | False |
| view_latest | 是否在初始化时滚动到最新对象 | ^[bool] | True |

### Slots

| 插槽名   | 说明               |
| ---     | ---               |
| default | 自定义默认内容      |

### 方法

| 方法名 | 说明 | 类型 |
| --- | --- | --- |
| send | 发送一个值并在聊天记录中创建一个新消息 | ^[Callable]`(value, user=None, avatar=None, respond=True, **kwargs) -> ChatMessage` |
| serialize | 将聊天记录导出为字典 | ^[Callable]`(format='transformers', role_names=None, default_role='user', filter_by=None, exclude_users=None, custom_serializer=None) -> Dict` |
| stream | 流式传输令牌并更新提供的消息 | ^[Callable]`(token, message=None, user=None, avatar=None, **kwargs) -> ChatMessage` |
| clear | 清除聊天记录并返回已清除的消息 | ^[Callable]`() -> List[ChatMessage]` |
| respond | 使用聊天记录中的最新消息执行回调 | ^[Callable]`() -> None` |
| trigger_post_hook | 使用聊天记录中的最新消息触发后钩子 | ^[Callable]`() -> None` |
| stop | 如果可能，取消当前回调任务 | ^[Callable]`() -> None` |
| scroll_to | 列滚动到指定索引处的对象 | ^[Callable]`(index: int) -> None` |
| undo | 从聊天记录中删除最后`count`条消息并返回它们 | ^[Callable]`(count: int = 1) -> List[ChatMessage]` |


## Controls

In [23]:
##controls
import panel as pn
pn.extension()

chat_feed = pn.chat.ChatFeed()
pn.Row(chat_feed.controls(jslink=False), chat_feed)

Row
    [0] Tabs
        [0] WidgetBox(margin=(5, 10), name='Controls')
            [0] StaticText(value='<b>Controls</b>')
            [1] ListInput(description='The list of c..., name='Objects', type=<class 'list'>)
            [2] Select(description='Whether to add scrollbars..., name='Scroll', options=OrderedDict([('False', ...]), value=False)
            [3] IntInput(description='Max pixel distance f..., name='Auto scroll limit', start=0, value=200)
            [4] LiteralInput(description='Callback to execute when ..., name='Callback')
            [5] LiteralInput(description='How to handle exceptions ..., name='Callback exception', value='summary')
            [6] TextInput(description='The default u..., name='Callback user', value='Assistant')
            [7] LiteralInput(description='The default avatar to use..., name='Callback avatar')
            [8] LiteralInput(description='Callback to execute when ..., name='Edit callback')
            [9] DictInput(description='Params to pass to Card, ..., name='Card params', type=<class 'dict'>)
            [10] Checkbox(name='Collapsible')
            [11] Checkbox(name='Disabled')
            [12] DictInput(description='Params to pass to each Ch..., name='Message params', type=<class 'dict'>)
            [13] LiteralInput(description='The header of the chat fe..., name='Header')
            [14] TextInput(description='If provided, ..., name='Help text')
            [15] IntInput(description='The number of objects loa..., name='Load buffer', start=0, value=50)
            [16] TextInput(description='The text to d..., name='Placeholder text')
            [17] DictInput(description='Params to pass to the pla..., name='Placeholder params', type=<class 'dict'>, value={'user': ' ', ...})
            [18] FloatInput(description='Min duration in seconds o..., name='Placeholder threshold', start=0, value=1)
            [19] LiteralInput(description='A hook to execute after a..., name='Post hook')
            [20] ListInput(description='A callable or list of cal..., name='Renderers', type=<class 'list'>)
            [21] IntInput(description='Min pixel distance f..., name='Scroll button threshold', start=0, value=100)
            [22] Checkbox(name='Show activity dot', value=True)
            [23] Checkbox(name='View latest', value=True)
            [24] Row(width=300)
                [0] LiteralInput(description='The placeholder w..., margin=(5, 0, 5, 10), name=' placeholder', sizing_mode='stretch_width', value=ChatMessage)
                [1] Toggle(align='end', button_type='primary', height_policy='fit', margin=(0, 0, 5, 10), max_height=30, max_width=20, name='⋮')
            [25] Select(description='The current s..., name=' callback state', options=OrderedDict([('IDLE', ...]), value=<CallbackState.IDLE: '...)
            [26] Button(description='Triggers the p..., name=' prompt trigger')
            [27] Button(description='Triggers the c..., name=' callback trigger')
            [28] ListInput(description='The previous d..., name=' disabled stack', type=<class 'list'>)
        [1] WidgetBox(margin=(5, 10), name='Layout')
            [0] StaticText(value='<b>Layout</b>')
            [1] TextInput(description='String identifier f..., name='Name', value='ChatFeed02981')
            [2] LiteralInput(description='Whether the object should..., name='Align', value='start')
            [3] LiteralInput(description='Describes the proportiona..., name='Aspect ratio')
            [4] ListInput(description='CSS classes t..., name='Css classes', type=<class 'list'>)
            [5] Select(description='The design system t..., name='Design', options=OrderedDict([('None', ...]))
            [6] IntInput(description='The height of the compone..., name='Height', start=0)
            [7] IntInput(description='Minimal width o..., name='Min width', start=0)
            [8] IntInput(description='Minimal height o..., name='Min height', start=0)
            [9] IntInput(description='Maximum width o..., n